In [1]:
import mediapipe as mp
import cv2
import numpy as np
import uuid
import os

Using drawing_utils to render the hand landmark model
<BR> Hands model:<img src=https://www.section.io/engineering-education/creating-a-hand-gesture-volume-controller-using-python-and-pycharm/hand.jpg>

In [2]:
mp_drawing = mp.solutions.drawing_utils
mp_hands = mp.solutions.hands

In [3]:
def draw_hlandmarks(frame,results):
    for num,hand in enumerate(results.multi_hand_landmarks):
                mp_drawing.draw_landmarks(frame,hand,mp_hands.HAND_CONNECTIONS) #rendering

Detection: Threshold for initial detecition to be successful
Tracking: Threshold for Tracking after Initial Detection

In [4]:
vidcap = cv2.VideoCapture(2)
with mp_hands.Hands(min_detection_confidence=0.8,min_tracking_confidence=0.5,max_num_hands=1) as hands:
    while vidcap.isOpened():
        ret,frame = vidcap.read()
        image = cv2.cvtColor(frame,cv2.COLOR_BGR2RGB)#OpenCV works in BlueGreenRed format but Mediapipe works in 
                                                     #RedGreenBlue format
        image.flags.writeable = False #setting flags to prevent copying the image
        results = hands.process(image) #detection
        image.flags.writeable = True #setting it back
        image = cv2.cvtColor(image,cv2.COLOR_RGB2BGR) #changing back
        #print(results) #output all the landmarks 
        if results.multi_hand_landmarks:
            draw_hlandmarks(frame,results)
        cv2.imshow('Hand Tracking',frame)
        if cv2.waitKey(10) & 0xFF == ord('q'):
            break
    vidcap.release()
    cv2.destroyAllWindows()

In [6]:
from matplotlib import pyplot as plt
draw_hlandmarks(frame,results)
plt.imshow(cv2.cvtColor(frame,cv2.COLOR_BGR2RGB))

NameError: name 'frame' is not defined

In [7]:
# np.save(os.path.join(DATA_PATH,action,str(sequence),str(frame_num)))

In [5]:
def extract_keypoints(results):
    if results.multi_hand_landmarks:
        return np.array([[lmark.x,lmark.y,lmark.z] for lmark in results.multi_hand_landmarks[0].landmark]).flatten() \
        if results.multi_hand_landmarks[0].landmark else np.zeros((21*3))
    else:
        return np.zeros((21*3))

In [9]:
extract_keypoints(results)

NameError: name 'results' is not defined

In [10]:
pose = extract_keypoints(results)
len(pose)

NameError: name 'results' is not defined

In [11]:
pose

NameError: name 'pose' is not defined

In [33]:
DATA_PATH= os.path.join('Mediapipe_Data') #Path for exported data
actions = np.array(['Active','Draw']) #Detect signs
no_sequences = 30 #number videos
sequences_length = 30 #number of frames

In [34]:
for action in actions:
    for sequence in range(no_sequences):
        try:
            os.makedirs(os.path.join(DATA_PATH,action,str(sequence)))
        except:
            pass

In [36]:
vidcap = cv2.VideoCapture(0)
with mp_hands.Hands(min_detection_confidence=0.8,min_tracking_confidence=0.5,max_num_hands=1) as hands:
    for action in actions:
        for sequence in range(no_sequences):
            for frame_num in range(sequences_length):
                ret,frame = vidcap.read()
                image = cv2.cvtColor(frame,cv2.COLOR_BGR2RGB)#OpenCV works in BlueGreenRed 
                                                             #format but Mediapipe works in RedGreenBlue format
                image.flags.writeable = False #setting flags to prevent copying the image
                results = hands.process(image) #detection
                image.flags.writeable = True #setting it back
                image = cv2.cvtColor(image,cv2.COLOR_RGB2BGR) #changing back
                #print(results) #output all the landmarks 
                if results.multi_hand_landmarks:
                    draw_hlandmarks(image,results) #rendering the landmarks
                if frame_num ==  0:
                    cv2.putText(image,'Start',(120,200),cv2.FONT_HERSHEY_SIMPLEX,1,(0,0,0),4,cv2.LINE_AA)
                    cv2.putText(image,'{} sequence {}'.format(action,sequence),(30,20),
                                cv2.FONT_HERSHEY_SIMPLEX,1,(0,0,0),1,cv2.LINE_AA)
                    cv2.imshow('Hand Tracking',image)
                    cv2.waitKey(2000) #two second delay between the videos
                else:
                    cv2.putText(image,'{} sequence {}'.format(action,sequence),(30,20),
                                cv2.FONT_HERSHEY_SIMPLEX,1,(0,0,0),1,cv2.LINE_AA)
                    cv2.imshow('Hand Tracking',image)
                
                
                keypoints = extract_keypoints(results)
                seqpath = os.path.join(DATA_PATH,action,str(sequence),str(frame_num))
                np.save(seqpath, keypoints)
                
                if cv2.waitKey(10) & 0xFF == ord('q'):
                    break
    vidcap.release()
    cv2.destroyAllWindows()

KeyboardInterrupt: 

In [37]:
vidcap.release()
cv2.destroyAllWindows()

In [38]:
from sklearn.model_selection import train_test_split
from tensorflow.keras.utils import to_categorical
labelmap = {label:num for num,label in enumerate(actions)}

In [39]:
labelmap

{'Active': 0, 'Draw': 1}

In [40]:
actions

array(['Active', 'Draw'], dtype='<U6')

In [41]:
sequences,labels = [],[]
for action in actions:
    for sequence in range(no_sequences):
        windows = []
        for frame_num in range(sequences_length):
            print(os.path.join(DATA_PATH,action,str(sequence),'{}.npy'.format(frame_num)))
            res = np.load(os.path.join(DATA_PATH,action,str(sequence),'{}.npy'.format(frame_num)))
            windows.append(res)
        sequences.append(windows)
        labels.append(labelmap[action])

Mediapipe_Data\Active\0\0.npy
Mediapipe_Data\Active\0\1.npy
Mediapipe_Data\Active\0\2.npy
Mediapipe_Data\Active\0\3.npy
Mediapipe_Data\Active\0\4.npy
Mediapipe_Data\Active\0\5.npy
Mediapipe_Data\Active\0\6.npy
Mediapipe_Data\Active\0\7.npy
Mediapipe_Data\Active\0\8.npy
Mediapipe_Data\Active\0\9.npy
Mediapipe_Data\Active\0\10.npy
Mediapipe_Data\Active\0\11.npy
Mediapipe_Data\Active\0\12.npy
Mediapipe_Data\Active\0\13.npy
Mediapipe_Data\Active\0\14.npy
Mediapipe_Data\Active\0\15.npy
Mediapipe_Data\Active\0\16.npy
Mediapipe_Data\Active\0\17.npy
Mediapipe_Data\Active\0\18.npy
Mediapipe_Data\Active\0\19.npy
Mediapipe_Data\Active\0\20.npy
Mediapipe_Data\Active\0\21.npy
Mediapipe_Data\Active\0\22.npy
Mediapipe_Data\Active\0\23.npy
Mediapipe_Data\Active\0\24.npy
Mediapipe_Data\Active\0\25.npy
Mediapipe_Data\Active\0\26.npy
Mediapipe_Data\Active\0\27.npy
Mediapipe_Data\Active\0\28.npy
Mediapipe_Data\Active\0\29.npy
Mediapipe_Data\Active\1\0.npy
Mediapipe_Data\Active\1\1.npy
Mediapipe_Data\Activ

Mediapipe_Data\Active\16\1.npy
Mediapipe_Data\Active\16\2.npy
Mediapipe_Data\Active\16\3.npy
Mediapipe_Data\Active\16\4.npy
Mediapipe_Data\Active\16\5.npy
Mediapipe_Data\Active\16\6.npy
Mediapipe_Data\Active\16\7.npy
Mediapipe_Data\Active\16\8.npy
Mediapipe_Data\Active\16\9.npy
Mediapipe_Data\Active\16\10.npy
Mediapipe_Data\Active\16\11.npy
Mediapipe_Data\Active\16\12.npy
Mediapipe_Data\Active\16\13.npy
Mediapipe_Data\Active\16\14.npy
Mediapipe_Data\Active\16\15.npy
Mediapipe_Data\Active\16\16.npy
Mediapipe_Data\Active\16\17.npy
Mediapipe_Data\Active\16\18.npy
Mediapipe_Data\Active\16\19.npy
Mediapipe_Data\Active\16\20.npy
Mediapipe_Data\Active\16\21.npy
Mediapipe_Data\Active\16\22.npy
Mediapipe_Data\Active\16\23.npy
Mediapipe_Data\Active\16\24.npy
Mediapipe_Data\Active\16\25.npy
Mediapipe_Data\Active\16\26.npy
Mediapipe_Data\Active\16\27.npy
Mediapipe_Data\Active\16\28.npy
Mediapipe_Data\Active\16\29.npy
Mediapipe_Data\Active\17\0.npy
Mediapipe_Data\Active\17\1.npy
Mediapipe_Data\Acti

Mediapipe_Data\Draw\1\20.npy
Mediapipe_Data\Draw\1\21.npy
Mediapipe_Data\Draw\1\22.npy
Mediapipe_Data\Draw\1\23.npy
Mediapipe_Data\Draw\1\24.npy
Mediapipe_Data\Draw\1\25.npy
Mediapipe_Data\Draw\1\26.npy
Mediapipe_Data\Draw\1\27.npy
Mediapipe_Data\Draw\1\28.npy
Mediapipe_Data\Draw\1\29.npy
Mediapipe_Data\Draw\2\0.npy
Mediapipe_Data\Draw\2\1.npy
Mediapipe_Data\Draw\2\2.npy
Mediapipe_Data\Draw\2\3.npy
Mediapipe_Data\Draw\2\4.npy
Mediapipe_Data\Draw\2\5.npy
Mediapipe_Data\Draw\2\6.npy
Mediapipe_Data\Draw\2\7.npy
Mediapipe_Data\Draw\2\8.npy
Mediapipe_Data\Draw\2\9.npy
Mediapipe_Data\Draw\2\10.npy
Mediapipe_Data\Draw\2\11.npy
Mediapipe_Data\Draw\2\12.npy
Mediapipe_Data\Draw\2\13.npy
Mediapipe_Data\Draw\2\14.npy
Mediapipe_Data\Draw\2\15.npy
Mediapipe_Data\Draw\2\16.npy
Mediapipe_Data\Draw\2\17.npy
Mediapipe_Data\Draw\2\18.npy
Mediapipe_Data\Draw\2\19.npy
Mediapipe_Data\Draw\2\20.npy
Mediapipe_Data\Draw\2\21.npy
Mediapipe_Data\Draw\2\22.npy
Mediapipe_Data\Draw\2\23.npy
Mediapipe_Data\Draw\2\24

Mediapipe_Data\Draw\22\1.npy
Mediapipe_Data\Draw\22\2.npy
Mediapipe_Data\Draw\22\3.npy
Mediapipe_Data\Draw\22\4.npy
Mediapipe_Data\Draw\22\5.npy
Mediapipe_Data\Draw\22\6.npy
Mediapipe_Data\Draw\22\7.npy
Mediapipe_Data\Draw\22\8.npy
Mediapipe_Data\Draw\22\9.npy
Mediapipe_Data\Draw\22\10.npy
Mediapipe_Data\Draw\22\11.npy
Mediapipe_Data\Draw\22\12.npy
Mediapipe_Data\Draw\22\13.npy
Mediapipe_Data\Draw\22\14.npy
Mediapipe_Data\Draw\22\15.npy
Mediapipe_Data\Draw\22\16.npy
Mediapipe_Data\Draw\22\17.npy
Mediapipe_Data\Draw\22\18.npy
Mediapipe_Data\Draw\22\19.npy
Mediapipe_Data\Draw\22\20.npy
Mediapipe_Data\Draw\22\21.npy
Mediapipe_Data\Draw\22\22.npy
Mediapipe_Data\Draw\22\23.npy
Mediapipe_Data\Draw\22\24.npy
Mediapipe_Data\Draw\22\25.npy
Mediapipe_Data\Draw\22\26.npy
Mediapipe_Data\Draw\22\27.npy
Mediapipe_Data\Draw\22\28.npy
Mediapipe_Data\Draw\22\29.npy
Mediapipe_Data\Draw\23\0.npy
Mediapipe_Data\Draw\23\1.npy
Mediapipe_Data\Draw\23\2.npy
Mediapipe_Data\Draw\23\3.npy
Mediapipe_Data\Draw\23\

In [42]:
len(sequences)

60

In [43]:
len(sequences[0][0])

63

In [44]:
np.array(sequences).shape

(60, 30, 63)

In [45]:
X_train, X_test, y_train, y_test = train_test_split(X,y,test_size=0.05)

In [46]:
X = np.array(sequences)
X.shape

(60, 30, 63)

In [47]:
y = to_categorical(labels).astype(int)

In [48]:
X_train, X_test, y_train, y_test = train_test_split(X,y,test_size=0.20)

In [49]:
y_test.shape

(12, 2)

In [50]:
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import LSTM,Dense
from tensorflow.keras.callbacks import TensorBoard

In [51]:
log_dir = os.path.join('logs')
tb_callback = TensorBoard(log_dir=log_dir)

In [86]:
model = Sequential()
model.add(LSTM(64,return_sequences=True, activation='relu', input_shape=(50,63)))
model.add(LSTM(128,return_sequences=True, activation='relu'))
model.add(LSTM(64,return_sequences=False, activation='relu'))
model.add(Dense(64, activation='relu'))
model.add(Dense(32, activation='relu'))
model.add(Dense(actions.shape[0]+1, activation='softmax'))

In [87]:
model.compile(optimizer='Adam',loss='categorical_crossentropy',metrics=['categorical_accuracy'])

In [57]:
model.fit(X_train,y_train,epochs=250,callbacks=[tb_callback])

Epoch 1/250
2/2 [==============================] - 8s 64ms/step - loss: 0.6968 - categorical_accuracy: 0.5625
Epoch 2/250
2/2 [==============================] - 0s 43ms/step - loss: 0.6869 - categorical_accuracy: 0.5625
Epoch 3/250
2/2 [==============================] - 0s 47ms/step - loss: 0.6849 - categorical_accuracy: 0.6875
Epoch 4/250
2/2 [==============================] - 0s 47ms/step - loss: 0.6766 - categorical_accuracy: 0.7708
Epoch 5/250
2/2 [==============================] - 0s 57ms/step - loss: 0.6619 - categorical_accuracy: 0.6667
Epoch 6/250
2/2 [==============================] - 0s 51ms/step - loss: 0.6355 - categorical_accuracy: 0.7500
Epoch 7/250
2/2 [==============================] - 0s 59ms/step - loss: 0.5628 - categorical_accuracy: 0.8542
Epoch 8/250
2/2 [==============================] - 0s 57ms/step - loss: 0.4501 - categorical_accuracy: 0.8333
Epoch 9/250
2/2 [==============================] - 0s 54ms/step - loss: 0.7353 - categorical_accuracy: 0.6458
Epoch 10/2

2/2 [==============================] - 0s 41ms/step - loss: 0.5131 - categorical_accuracy: 0.7292
Epoch 75/250
2/2 [==============================] - 0s 42ms/step - loss: 0.5141 - categorical_accuracy: 0.7292
Epoch 76/250
2/2 [==============================] - 0s 41ms/step - loss: 0.5109 - categorical_accuracy: 0.7292
Epoch 77/250
2/2 [==============================] - 0s 44ms/step - loss: 0.5036 - categorical_accuracy: 0.7292
Epoch 78/250
2/2 [==============================] - 0s 40ms/step - loss: 0.4878 - categorical_accuracy: 0.7292
Epoch 79/250
2/2 [==============================] - 0s 41ms/step - loss: 0.4638 - categorical_accuracy: 0.7500
Epoch 80/250
2/2 [==============================] - 0s 239ms/step - loss: 0.4410 - categorical_accuracy: 0.7500
Epoch 81/250
2/2 [==============================] - 0s 39ms/step - loss: 0.4222 - categorical_accuracy: 0.7917
Epoch 82/250
2/2 [==============================] - 0s 43ms/step - loss: 0.3943 - categorical_accuracy: 0.7917
Epoch 83/250


2/2 [==============================] - 0s 44ms/step - loss: 0.0521 - categorical_accuracy: 1.0000
Epoch 148/250
2/2 [==============================] - 0s 44ms/step - loss: 0.0509 - categorical_accuracy: 1.0000
Epoch 149/250
2/2 [==============================] - 0s 41ms/step - loss: 0.0493 - categorical_accuracy: 1.0000
Epoch 150/250
2/2 [==============================] - 0s 41ms/step - loss: 0.0476 - categorical_accuracy: 1.0000
Epoch 151/250
2/2 [==============================] - 0s 41ms/step - loss: 0.0462 - categorical_accuracy: 1.0000
Epoch 152/250
2/2 [==============================] - 0s 51ms/step - loss: 0.0444 - categorical_accuracy: 1.0000
Epoch 153/250
2/2 [==============================] - 0s 50ms/step - loss: 0.0428 - categorical_accuracy: 1.0000
Epoch 154/250
2/2 [==============================] - 0s 44ms/step - loss: 0.0410 - categorical_accuracy: 1.0000
Epoch 155/250
2/2 [==============================] - 0s 39ms/step - loss: 0.0390 - categorical_accuracy: 1.0000
Epoch 

KeyboardInterrupt: 

In [58]:
 res = model.predict(X_test)

1/1 [==============================] - 1s 756ms/step


In [59]:
y_test.shape

(12, 2)

In [60]:
actions[np.argmax(y_test[0])]

'Draw'

In [61]:
model.save('actions1.h5')

In [88]:
model.load_weights('gesture.h5')

In [76]:
from sklearn.metrics import multilabel_confusion_matrix, accuracy_score
yhat = model.predict(X_test)
yhat = np.argmax(yhat, axis=1).tolist()
ytrue = np.argmax(y_test,axis=1).tolist()

1/1 [==============================] - 0s 44ms/step


In [77]:
actions[np.argmax(res[0])]

'Active'

In [78]:
res[0][np.argmax(res[0])]

IndexError: invalid index to scalar variable.

In [79]:
multilabel_confusion_matrix(ytrue,yhat)

array([[[1, 8],
        [0, 3]],

       [[3, 0],
        [8, 1]]], dtype=int64)

In [80]:
accuracy_score(ytrue,yhat)

0.3333333333333333

In [92]:
#Testing in Real Time
sequence = []
sentence = []
threshold = 0.9 #90% 
#res = model.predict(X_test)
vidcap = cv2.VideoCapture(0)
with mp_hands.Hands(min_detection_confidence=0.8,min_tracking_confidence=0.5,max_num_hands=1) as hands:
    while vidcap.isOpened():
        ret,frame = vidcap.read()
        image = cv2.cvtColor(frame,cv2.COLOR_BGR2RGB)#OpenCV works in BlueGreenRed format 
                                                     #but Mediapipe works in RedGreenBlue format
        image.flags.writeable = False #setting flags to prevent copying the image
        results = hands.process(image) #detection
        image.flags.writeable = True #setting it back
        image = cv2.cvtColor(image,cv2.COLOR_RGB2BGR) #changing back
        #print(results) #output all the landmarks 
        if results.multi_hand_landmarks:
            draw_hlandmarks(frame,results) #rendering
        
        #predictions
        keypoints = extract_keypoints(results)
        sequence.append(keypoints)
        sequence = sequence[-50:]
        
        if len(sequence) == 50:
            res = model.predict(np.expand_dims(sequence,axis=0))[0] #get the result
            print(actions[np.argmax(res)]) 
            
            if res[np.argmax(res)] > threshold:
                if len(sentence) > 0:
                    if actions[np.argmax(res)] != sentence[-1]:
                        sentence.append(actions[np.argmax(res)])
                else:
                    sentence.append(actions[np.argmax(res)])

        if len(sentence) > 5:
            sentence = sentence[-5:0]
        cv2.rectangle(frame,(0,0),(640,40),(245,117,16),-1)
        cv2.putText(frame,' '.join(sentence),(3,30),cv2.FONT_HERSHEY_SIMPLEX,1,(255,255,255),2,cv2.LINE_AA)
        
        cv2.imshow('Hand Tracking',frame)
        if cv2.waitKey(10) & 0xFF == ord('q'):
            break
    vidcap.release()
    cv2.destroyAllWindows()

1/1 [==============================] - 0s 37ms/step


IndexError: index 2 is out of bounds for axis 0 with size 2

In [91]:
vidcap.release()
cv2.destroyAllWindows()

In [32]:
#Testing in Real Time
sequence = []
result = "" 
threshold = 0.40 #90% 
res = model.predict(X_test)
vidcap = cv2.VideoCapture(0)
with mp_hands.Hands(min_detection_confidence=0.8,min_tracking_confidence=0.5,max_num_hands=1) as hands:
    while vidcap.isOpened():
        ret,frame = vidcap.read()
        image = cv2.cvtColor(frame,cv2.COLOR_BGR2RGB)#OpenCV works in BlueGreenRed format 
                                                     #but Mediapipe works in RedGreenBlue format
        image.flags.writeable = False #setting flags to prevent copying the image
        results = hands.process(image) #detection
        image.flags.writeable = True #setting it back
        image = cv2.cvtColor(image,cv2.COLOR_RGB2BGR) #changing back
        #print(results) #output all the landmarks 
        if results.multi_hand_landmarks:
            draw_hlandmarks(frame,results) #rendering
        
        #predictions
        keypoints = extract_keypoints(results)
        sequence.append(keypoints)
        sequence = sequence[-50:]
        
        if len(sequence) == 50:
            res = model.predict(np.expand_dims(sequence,axis=0))[0] #get the result
            if res[np.argmax(res)] > threshold:
                result = actions[np.argmax(res)]
        cv2.rectangle(frame,(0,0),(640,40),(245,117,16),-1)
        cv2.putText(frame,result,(3,30),cv2.FONT_HERSHEY_SIMPLEX,1,(255,255,255),2,cv2.LINE_AA)
        
        cv2.imshow('Hand Tracking',frame)
        if cv2.waitKey(10) & 0xFF == ord('q'):
            break
    vidcap.release()
    cv2.destroyAllWindows()

1/1 [==============================] - 0s 35ms/step
